## Data Analysis and Manipulation

### Part - 1

#### Setting the Postgres Sql database in Render and Creating the tables with Movies and Ratings names and also loading the data into it with the provided CSV files. 

In [53]:
# Loading the important liberaries required for establishing the connection and loading the data

In [143]:
import pandas as pd
import psycopg2
from psycopg2 import sql
from sqlalchemy import create_engine

In [113]:
connection_params = {
    'host': 'dpg-cm0t24ocmk4c739e1fi0-a.oregon-postgres.render.com',
    'port': 5432,
    'user': 'squadcast_assignment_database_user',
    'password': 'ofMkgeev2bS24Zd3FONzt9lhbIqfTzjl',
    'database': 'squadcast_assignment_database',
}

connection = psycopg2.connect(**connection_params)
cursor = connection.cursor()

movies_file_path = 'D:\\Squadcast Assignment\\movies.csv'
ratings_file_path = 'D:\\Squadcast Assignment\\ratings.csv'


engine = create_engine(f'postgresql+psycopg2://{connection_params["user"]}:{connection_params["password"]}@{connection_params["host"]}:{connection_params["port"]}/{connection_params["database"]}')


movies_table = """
CREATE TABLE IF NOT EXISTS movies (
    id SERIAL PRIMARY KEY,
    title VARCHAR(50),
    year INTEGER,
    country VARCHAR(50),
    genre VARCHAR(50),
    director VARCHAR(50),
    minutes INTEGER,
    poster VARCHAR(500)
);
"""

ratings_table = """
CREATE TABLE IF NOT EXISTS ratings (
    rater_id INTEGER,
    movie_id INTEGER,
    rating INTEGER,
    time INTEGER
);
"""

with engine.connect() as connection:
    connection.execute(movies_table)
    connection.execute(ratings_table)

movies_df = pd.read_csv(movies_file_path)
ratings_df = pd.read_csv(ratings_file_path)

movies_df.to_sql('movies', con=engine, if_exists='replace', index=False)
ratings_df.to_sql('ratings', con=engine, if_exists = 'replace',index = False)

1000

In [56]:
# The above steps shows that data is being perfectly loaded into Postgres database

In [114]:
movies_df.head()

,id,title,year,country,genre,director,minutes,poster
0,3112654,Mea culpa,2014,France,"Action, Thriller",Fred Cavayé,90,http://ia.media-imdb.com/images/M/MV5BNTQ0ODAw...
1,1571222,A Dangerous Method,2011,"UK, Germany, Canada, Switzerland","Biography, Drama, Thriller",David Cronenberg,99,http://ia.media-imdb.com/images/M/MV5BMTU5Mjk3...
2,2756032,The One I Love,2014,USA,"Comedy, Drama, Romance",Charlie McDowell,91,http://ia.media-imdb.com/images/M/MV5BMTgzODMy...
3,2370248,Short Term 12,2013,USA,Drama,Destin Daniel Cretton,96,http://ia.media-imdb.com/images/M/MV5BMTEwNjE2...
4,57171,"Yesterday, Today and Tomorrow",1963,"Italy, France","Comedy, Romance",Vittorio De Sica,119,http://ia.media-imdb.com/images/M/MV5BMTY0NDc4...


In [115]:
movies_df.shape

(3143, 8)

In [116]:
movies_df.columns

Index(['id', 'title', 'year', 'country', 'genre', 'director', 'minutes',
       'poster'],
      dtype='object')

In [117]:
movies_df.dtypes

id           int64
title       object
year         int64
country     object
genre       object
director    object
minutes      int64
poster      object
dtype: object

In [118]:
ratings_df.head()

,rater_id,movie_id,rating,time
0,1,68646,10,1381620027
1,1,113277,10,1379466669
2,2,422720,8,1412178746
3,2,454876,8,1394818630
4,2,790636,7,1389963947


In [119]:
ratings_df.shape

(10000, 4)

In [120]:
ratings_df.dtypes

rater_id    int64
movie_id    int64
rating      int64
time        int64
dtype: object

In [61]:
# movies file contains 3143 rows and 8 columns 

In [68]:
# ratings files contains 10000 rows and 4 columns

In [98]:
cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public';")
tables = cursor.fetchall()
tables

[('movies',), ('ratings',)]

### Part - 2 (Insights and Analysis)

###### Top 5 Movie Titles: Sort and print the top 5 movie titles based on the following criteria:

1. Duration
2. Year of Release
3. Average rating (consider movies with minimum 5 ratings)
4. Number of ratings given

In [153]:
combined_df = pd.merge(movies_df,ratings_df,left_on='id',right_on='movie_id',how = 'right')

In [194]:
combined_df.head()

,id,title,year,country,genre,director,minutes,poster,rater_id,movie_id,rating,time
0,68646,The Godfather,1972,USA,"Crime, Drama",Francis Ford Coppola,175,http://ia.media-imdb.com/images/M/MV5BMjEyMjcy...,1,68646,10,1381620027
1,113277,Heat,1995,USA,"Action, Crime, Drama",Michael Mann,170,http://ia.media-imdb.com/images/M/MV5BMTM1NDc4...,1,113277,10,1379466669
2,422720,Marie Antoinette,2006,"USA, France, Japan","Biography, Drama, History",Sofia Coppola,123,http://ia.media-imdb.com/images/M/MV5BMTIzOTk1...,2,422720,8,1412178746
3,454876,Life of Pi,2012,"USA, Taiwan, UK","Adventure, Drama, Fantasy",Ang Lee,127,http://ia.media-imdb.com/images/M/MV5BNTg2OTY2...,2,454876,8,1394818630
4,790636,Dallas Buyers Club,2013,USA,"Biography, Drama",Jean-Marc Vallée,117,NaN,2,790636,7,1389963947


In [198]:
top_movies_by_duration = movies_df.sort_values(by='minutes', ascending=False).head(5)[['title', 'minutes']]

In [197]:
top_movies_by_duration

,title,minutes
903,Gangs of Wasseypur,320
1065,Gone with the Wind,238
157,Once Upon a Time in America,229
434,Lawrence of Arabia,216
2595,Jodhaa Akbar,213


In [ ]:
#Top movies by duration are Gangs of Wasseypur with the highest 320 minutes, Gone with the wind, Once upon a time in America, Lawerence of Arabia, jodhaa akbhar

In [199]:
top_movies_by_year = movies_df.sort_values(by='year', ascending=False).head(5)[['title', 'year']]
top_movies_by_year

,title,year
1855,Maggie,2015
2142,Tracers,2015
1157,Hidden,2015
2320,They Look Like People,2015
1145,Hitman: Agent 47,2015


In [ ]:
# Top five movies based on year are Maggie, Tracers, Hidden, They look like People and Hitman: Agent 47 they got released around year 2015

In [200]:
# Consider movies with minimum 5 ratings

In [201]:
avg_ratings = ratings_df.groupby('movie_id')['rating'].mean().reset_index()
avg_ratings = avg_ratings[avg_ratings['rating'] >= 5]
avg_ratings = pd.merge(avg_ratings, movies_df, left_on='movie_id', right_on='id', how='inner')
top_movies_by_avg_rating = avg_ratings.sort_values(by='rating', ascending=False).head(5)[['title', 'rating']]
top_movies_by_avg_rating

,title,rating
531,New Nightmare,10.0
286,Conan the Barbarian,10.0
681,U.S. Marshals,10.0
1359,Seven Pounds,10.0
2002,The Prophet,10.0


In [202]:
#the above movies have avg ratings of New Nighmare, conan the barbarian, U.S Marshals, Seven pounds The Prophet have the avg ratings of 10

In [203]:
#Number of ratings given

In [204]:
num_ratings = ratings_df.groupby('movie_id')['rating'].count().reset_index()
num_ratings = pd.merge(num_ratings, movies_df, left_on='movie_id', right_on='id', how='inner')
top_movies_by_num_ratings = num_ratings.sort_values(by='rating', ascending=False).head(5)[['title', 'rating']]
top_movies_by_num_ratings

,title,rating
1522,The Wolf of Wall Street,69
1917,Gravity,64
1365,Man of Steel,60
2143,Now You See Me,59
1412,Interstellar,58


In [205]:
# the wolf of the wall street has highest number ratings of 69, Gravity 64, Man of Steel 60, Now you see me 59, Interstellar 58

#### B. Number of Unique Raters: Determine and print the count of unique rater IDs.

In [206]:
unique_raters_count = ratings_df['rater_id'].nunique()
unique_raters_count

1048

In [207]:
# 1048 are the numbers of the unique_raters_count

##### C. Top 5 Rater IDs: Sort and print the top 5 rater IDs based on:
1. Most movies rated
2. Highest Average rating given (consider raters with min 5 ratings) 

In [210]:
top_raters_by_movies_rated = ratings_df['rater_id'].value_counts().head(5)
pd.DataFrame(top_raters_by_movies_rated)

,count
rater_id,
735,314
769,209
512,188
800,186
82,159


In [ ]:
# The Person with rater id 735 has rated of more 314 times 

In [211]:
avg_ratings_by_rater = ratings_df.groupby('rater_id')['rating'].mean().reset_index()
avg_ratings_by_rater = avg_ratings_by_rater[avg_ratings_by_rater['rating'] >= 5]
top_raters_by_avg_rating = avg_ratings_by_rater.sort_values(by='rating', ascending=False).head(5)
top_raters_by_avg_rating

,rater_id,rating
0,1,10.0
217,218,10.0
441,442,10.0
439,440,10.0
438,439,10.0


In [212]:
# Raters IDs with 1,218,442,440,439 have given highest average rater id

##### D. Top Rated Movie
- Find and print the top-rated movies by:

    1. Director 'Michael Bay',
    2.'Comedy',
    3. In the year 2013
    4. In India (consider movies with a minimum of 5 ratings).

In [215]:
action_movies_criteria = (
    (ratings_df['rating'] >= 5) &
    ((movies_df['director'] == 'Michael Bay') |
    (movies_df['genre'] == 'Comedy') |
    (movies_df['year'] == 2013) |
    (movies_df['country'] == 'India'))
)

top_rated_movies = pd.merge(ratings_df[action_movies_criteria], movies_df, left_on='movie_id', right_on='id', how='inner')
top_rated_movies = top_rated_movies.groupby('title')['rating'].mean().reset_index()
top_rated_movies = top_rated_movies.sort_values(by='rating', ascending=False).head(5)[['title', 'rating']]
top_rated_movies

,title,rating
69,Cool Hand Luke,10.0
46,Byzantium,10.0
320,The Godfather: Part II,10.0
323,The Great Debaters,10.0
252,Savages,10.0


In [216]:
# Movies Names with cool Hand Luke, Byzantium, the Godfather: part 2, the great Debaters, savages have the avg ratings of 10

#### E.  Favorite Movie Genre of Rater ID 1040: Determine and print the favorite movie genre for the rater with ID 1040 (defined as the genre of the movie the rater has rated most often).

In [221]:
ratings_by_rater_1040 = combined_df[combined_df['rater_id'] == 1040]
favorite_genre_1040 = ratings_by_rater_1040['genre'].value_counts().idxmax()
favorite_genre_1040

'Action, Adventure, Sci-Fi'

In [222]:
# The Favorite movie genre of the rater_id 1040 is Action, Adventure, Sci-Fi

##### F. Highest Average Rating for a Movie Genre by Rater ID 1040: Find and print the highest average rating for a movie genre given by the rater with ID 1040 (consider genres with a minimum of 5 ratings).

In [224]:
genres_with_min_5_ratings_1040 = ratings_by_rater_1040['genre'].value_counts().index[ratings_by_rater_1040['genre'].value_counts() >= 5]
ratings_by_rater_1040_filtered = ratings_by_rater_1040[ratings_by_rater_1040['genre'].isin(genres_with_min_5_ratings_1040)]
highest_avg_rating_genre_1040 = ratings_by_rater_1040_filtered.groupby('genre')['rating'].mean().idxmax()
highest_avg_rating_1040 = ratings_by_rater_1040_filtered.groupby('genre')['rating'].mean().max()
highest_avg_rating_1040

8.454545454545455

In [225]:
# the Highest average ratings of movie genre vy rater id 1040 is 8.45

##### G.Year with Second-Highest Number of Action Movies: Identify and print the year with the second-highest number of action movies from the USA that received an average rating of 6.5 or higher and had a runtime of less than 120 minutes.

In [239]:
action_movies_criteria = (
    (combined_df['genre'] == 'Action') &
    (combined_df['country'] == 'USA') &
    (combined_df['rating'] >= 6.5) &
    (combined_df['minutes'] < 120)
)
action_movies_filtered = combined_df[action_movies_criteria]
if len(action_movies_filtered['year'].unique()) >= 2:
    action_movies_count_by_year = action_movies_filtered['year'].value_counts()
    second_highest_year_action_movies = action_movies_count_by_year.index[1]
    print("Second highest year with action movies:", second_highest_year_action_movies)
else:
    print("There are not enough years with action movies to determine the second-highest year.")


There are not enough years with action movies to determine the second-highest year.


##### H. Count of Movies with High Ratings: Count and print the number of movies that have received at least five reviews with a rating of 7 or higher.

In [240]:
high_rated_movies_count = pd.merge(ratings_df, movies_df, left_on='movie_id', right_on='id', how='inner')
high_rated_movies_count = high_rated_movies_count.groupby('title').filter(lambda x: len(x) >= 5 and x['rating'].mean() >= 7)['title'].nunique()
high_rated_movies_count

305

In [242]:
# 305 movies are with highest movie ratings in five reviews with 5 ratings

In [244]:
# Dispalying the results

In [245]:
print("a. Top 5 Movie Titles by Duration:")
print(top_movies_by_duration)

print("\nb. Number of Unique Raters:", unique_raters_count)

print("\nc. Top 5 Rater IDs by Most Movies Rated:")
print(top_raters_by_movies_rated)

print("\nd. Top Rated Movies:")
print(top_rated_movies)

print("\ne. Favorite Movie Genre of Rater ID 1040:", favorite_genre_1040)

print("\nf. Highest Average Rating for a Movie Genre by Rater ID 1040:")
print(f"Genre '{highest_avg_rating_genre_1040}' with a rating of {highest_avg_rating_1040:.2f}")

print("\ng. Year with Second-Highest Number of Action Movies:", second_highest_year_action_movies)

print("\nh. Count of Movies with High Ratings:", high_rated_movies_count)

a. Top 5 Movie Titles by Duration:
                            title  minutes
903            Gangs of Wasseypur      320
1065           Gone with the Wind      238
157   Once Upon a Time in America      229
434            Lawrence of Arabia      216
2595                 Jodhaa Akbar      213

b. Number of Unique Raters: 1048

c. Top 5 Rater IDs by Most Movies Rated:
rater_id
735    314
769    209
512    188
800    186
82     159
Name: count, dtype: int64

d. Top Rated Movies:
                      title  rating
69           Cool Hand Luke    10.0
46                Byzantium    10.0
320  The Godfather: Part II    10.0
323      The Great Debaters    10.0
252                 Savages    10.0

e. Favorite Movie Genre of Rater ID 1040: Action, Adventure, Sci-Fi

f. Highest Average Rating for a Movie Genre by Rater ID 1040:
Genre 'Action, Adventure, Sci-Fi' with a rating of 8.45

g. Year with Second-Highest Number of Action Movies: year
2003    5
Name: count, dtype: int64

h. Count of Movies 